In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas_datareader as pdr

Question 1 
------------------------------------------------------------------------------

In [2]:
# OLD DATA LOADING
"""

data = pd.read_csv("data/25_Portfolios_5x5.csv",skiprows=15,index_col="DATE",parse_dates=True,nrows=1134)
data = data.dropna()
data.index = pd.to_datetime(data.index,format="%Y%m")
factors = pd.read_csv("data/F-F_Research_Data_Factors.csv",skiprows=3,index_col=0,parse_dates=True,nrows=1134)
factors = factors.dropna()
factors.index = pd.to_datetime(factors.index,format="%Y%m")
rm = factors["Mkt-RF"]
rf = factors["RF"]
"""
data = pdr.get_data_famafrench("25_Portfolios_5x5", start="1963-07", end="2020-06")[0]
data.rename(columns={"SMALL LoBM": "ME1 BM1", "SMALL HiBM": "ME1 BM5","BIG LoBM": "ME5 BM1", "BIG HiBM": "ME5 BM5"}, inplace=True)
factors = pdr.get_data_famafrench("F-F_Research_Data_Factors", start="1963-07", end="2020-06")[0]
factors = factors.drop(columns=["SMB", "HML"])
rf = factors["RF"]
rm = factors["Mkt-RF"]
factors



,Mkt-RF,RF
Date,,
1963-07,-0.39,0.27
1963-08,5.07,0.25
1963-09,-1.57,0.27
1963-10,2.53,0.29
1963-11,-0.85,0.27
...,...,...
2020-02,-8.13,0.12
2020-03,-13.38,0.12
2020-04,13.65,0.00


In [3]:
rm

Date
1963-07    -0.39
1963-08     5.07
1963-09    -1.57
1963-10     2.53
1963-11    -0.85
           ...  
2020-02    -8.13
2020-03   -13.38
2020-04    13.65
2020-05     5.58
2020-06     2.46
Freq: M, Name: Mkt-RF, Length: 684, dtype: float64

data = data["1963-07-01":"2020-06-01"]
rm = rm["1963-07-01":"2020-06-01"]
rf = rf["1963-07-01":"2020-06-01"]

In [4]:
arith_mean = data.mean(0)
simple_mean = pd.DataFrame(np.zeros((5,5)),index=["SMALL","ME2","ME3","ME4","BIG"],columns=["LoBM","BM2","BM3","BM4","HiBM"])
for r in range(0,5):
    for c in range(0,5):
        n = r * 5 + c
        simple_mean.iloc[r,c] = arith_mean[n]
simple_mean.round(2)

,LoBM,BM2,BM3,BM4,HiBM
SMALL,0.67,1.18,1.13,1.32,1.38
ME2,0.89,1.14,1.22,1.22,1.30
ME3,0.92,1.15,1.11,1.20,1.32
ME4,1.02,0.99,1.04,1.17,1.18
BIG,0.92,0.89,0.93,0.85,0.97


In [5]:
excess_data = data.subtract(rf,axis=0)
excess_data

,ME1 BM1,ME1 BM2,ME1 BM3,ME1 BM4,ME1 BM5,ME2 BM1,ME2 BM2,ME2 BM3,ME2 BM4,ME2 BM5,...,ME4 BM1,ME4 BM2,ME4 BM3,ME4 BM4,ME4 BM5,ME5 BM1,ME5 BM2,ME5 BM3,ME5 BM4,ME5 BM5
Date,,,,,,,,,,,,,,,,,,,,,
1963-07,0.8607,-0.6319,0.4652,-0.1603,-1.5912,-2.0771,-0.0801,-1.1145,-2.2344,-1.4589,...,-1.1156,-2.0435,-2.1884,-1.9357,-2.1267,-0.1299,0.2133,0.9586,-0.8562,-1.3726
1963-08,3.9870,1.1213,1.1474,2.1201,4.4245,5.3165,4.2691,4.1139,4.1688,7.9829,...,5.1465,4.4911,5.9832,7.3119,5.0969,5.5210,4.0050,4.3436,8.0331,6.1324
1963-09,-3.1578,0.3473,-1.3477,-1.8512,-0.6281,-4.3202,-1.7734,-0.9117,-1.4512,-3.1849,...,-2.9593,-2.3753,-2.0500,-3.8523,-2.2643,-1.6421,-1.0754,-1.0835,-0.4845,-3.7663
1963-10,0.9985,-1.0213,1.0704,-0.1900,2.0824,0.9016,3.9442,2.0538,1.9112,3.6435,...,-0.7187,0.3889,2.3329,5.0482,0.3213,5.0361,1.4527,-0.5369,2.1015,0.1957
1963-11,-3.6451,-3.9636,-2.0452,-1.3248,-1.3154,-4.5261,-2.0234,-0.9764,-0.3702,-0.3832,...,-1.1348,-0.9003,-1.0633,0.8751,3.2688,-1.5261,0.7372,-2.0125,-2.3538,1.0755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02,-6.2492,-4.3556,-4.5121,-8.5120,-8.6727,-7.6318,-8.4373,-10.9698,-10.4070,-10.6987,...,-7.0489,-7.8165,-9.2868,-10.9675,-13.5485,-7.0942,-8.0794,-9.1973,-10.8881,-11.8301
2020-03,-21.6093,-15.2902,-22.1317,-26.6655,-28.0484,-19.9826,-19.4271,-21.4952,-23.9676,-32.3970,...,-12.4341,-19.4122,-20.4564,-31.5469,-32.9224,-7.8616,-11.3201,-15.9014,-20.2342,-28.5209
2020-04,24.4356,20.9895,15.1668,10.9867,15.8564,19.2871,12.6890,12.0894,9.9204,26.2566,...,16.7869,13.9499,11.9545,16.2333,17.7799,14.0851,13.6411,9.8370,10.4612,15.5348


In [6]:
betas = {}
for column in excess_data:
    res = sm.OLS(excess_data[column],rm,cov_type="HC0").fit()
    betas[column] = res.params
betas = pd.DataFrame(betas).T

betas = betas.squeeze()

In [7]:
beta_table = pd.DataFrame(np.zeros((5,5)),index=["SMALL","ME2","ME3","ME4","BIG"],columns=["LoBM","BM2","BM3","BM4","HiBM"])
for r in range(0,5):
    for c in range(0,5):
        n = r * 5 + c
        beta_table.iloc[r,c] = betas[n]
beta_table.round(2)

,LoBM,BM2,BM3,BM4,HiBM
SMALL,1.42,1.24,1.12,1.06,1.09
ME2,1.39,1.19,1.08,1.03,1.17
ME3,1.32,1.13,1.02,1.00,1.09
ME4,1.22,1.09,1.01,1.00,1.11
BIG,0.98,0.93,0.86,0.89,0.98


In [8]:
# Standard deviation
stds = data.std(0)
std_dev = pd.DataFrame(np.zeros((5,5)),index=["SMALL","ME2","ME3","ME4","BIG"],columns=["LoBM","BM2","BM3","BM4","HiBM"])
for r in range(0,5):
    for c in range(0,5):
        n = r * 5 + c
        std_dev.iloc[r,c] = stds[n]
#print(simple_mean.to_latex())
std_dev.round(2)

,LoBM,BM2,BM3,BM4,HiBM
SMALL,7.93,6.94,6.02,5.80,6.05
ME2,7.15,6.01,5.48,5.32,6.19
ME3,6.56,5.50,5.02,5.04,5.79
ME4,5.85,5.16,4.95,5.00,5.79
BIG,4.60,4.40,4.29,4.72,5.56


In [9]:
# Sharpe ratio
sharpe = ((simple_mean - rf.mean())/std_dev) * np.sqrt(12)
sharpe

,LoBM,BM2,BM3,BM4,HiBM
SMALL,0.127981,0.398980,0.432316,0.561455,0.574869
ME2,0.247926,0.440792,0.531493,0.547959,0.518837
ME3,0.287120,0.485512,0.508304,0.568969,0.564580
ME4,0.383910,0.415031,0.465588,0.549692,0.484319
BIG,0.413268,0.407632,0.450247,0.346043,0.369699


In [10]:
nobs = data.shape[0]
tstats = (simple_mean / (std_dev/np.sqrt(nobs)) )
tstats.round(2)

,LoBM,BM2,BM3,BM4,HiBM
SMALL,2.21,4.43,4.90,5.93,5.97
ME2,3.25,4.96,5.81,5.98,5.50
ME3,3.67,5.45,5.80,6.25,5.96
ME4,4.58,5.04,5.50,6.11,5.35
BIG,5.26,5.31,5.69,4.70,4.56


In [11]:
long_short_row = {}
for i in range(4,25,5):
    long_short_row[i] = data.iloc[:,i] - data.iloc[:,i-4]
long_short_row = pd.DataFrame(long_short_row)
lsr_tstat = (long_short_row.mean() / (long_short_row.std() / np.sqrt(nobs))) 

lsr_tstat = pd.DataFrame(lsr_tstat,columns=["Long Short"])
lsr_tstat.index = ["SMALL","ME2","ME3","ME4","BIG"]

tstats["Long Short"] = lsr_tstat
tstats

,LoBM,BM2,BM3,BM4,HiBM,Long Short
SMALL,2.206048,4.427731,4.896276,5.932421,5.965332,4.520056
ME2,3.245429,4.961949,5.807011,5.983933,5.504287,2.700108
ME3,3.665573,5.450730,5.795338,6.245003,5.959252,2.440588
ME4,4.576893,5.037013,5.501178,6.113798,5.354226,0.994514
BIG,5.256091,5.311633,5.691276,4.695173,4.556936,0.274618


In [12]:
long_short_col = {}
for i in range(20,25):
    long_short_col[i] = data.iloc[:,i] - data.iloc[:,i-20]

long_short_col = pd.DataFrame(long_short_col)
lsc_tstat = (long_short_col.mean() / (long_short_col.std() / np.sqrt(nobs))) 

lsc_tstat = pd.DataFrame(lsc_tstat,columns=["Long Short"]).T
lsc_tstat.columns = ["LoBM","BM2","BM3","BM4","HiBM"]

#tstats["Long Short"] = lsc_tstat
tstats_extended = tstats.append(lsc_tstat)
tstats_extended.round(2)

,LoBM,BM2,BM3,BM4,HiBM,Long Short
SMALL,2.21,4.43,4.90,5.93,5.97,4.52
ME2,3.25,4.96,5.81,5.98,5.50,2.70
ME3,3.67,5.45,5.80,6.25,5.96,2.44
ME4,4.58,5.04,5.50,6.11,5.35,0.99
BIG,5.26,5.31,5.69,4.70,4.56,0.27
Long Short,1.17,-1.42,-1.13,-2.78,-2.27,NaN
